In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from dataset import create_QM9_train_val_test_nx

In [14]:
train_graphs_subset, val_graphs_subset, test_graphs_subset, \
train_pyg_subset, val_pyg_subset, test_pyg_subset = create_QM9_train_val_test_nx(
    subset_size=None,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    random_seed=42
)

Original dataset size: 130831
Train NX graphs: 91581, Train PyG graphs: 91581
Validation NX graphs: 19624, Validation PyG graphs: 19624
Test NX graphs: 19626, Test PyG graphs: 19626


In [12]:
# Custom Graph2Vec implementation to avoid KarateClub dependency issues
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import PCA

def custom_graph2vec(nx_graphs, wl_iterations=3, embedding_dim=64):
    """
    Custom Graph2Vec implementation using Weisfeiler-Lehman kernel
    
    Args:
        graphs: List of networkx graphs
        wl_iterations: Number of WL iterations
        embedding_dim: Dimension of resulting embeddings
        
    Returns:
        numpy array with graph embeddings
    """
        
    # Initialize with node labels (use degrees as initial labels)
    documents = []
    node_labels = [{node: str(nx_graphs[i].degree(node)) for node in nx_graphs[i].nodes()}
                    for i in range(len(nx_graphs))]
    
    # WL iterations
    for _ in range(wl_iterations):
        # For each graph, create a document of vertex labels
        graph_documents = []
        new_node_labels = []
        
        for i, g in enumerate(nx_graphs):
            labels = node_labels[i]
            new_labels = {}
            doc = []
            
            # For each node, aggregate neighbor labels
            for node in g.nodes():
                neighbors = sorted([labels[nbr] for nbr in g.neighbors(node)])
                new_label = f"{labels[node]}_{'_'.join(neighbors)}"
                new_labels[node] = new_label
                doc.append(new_label)
                
            new_node_labels.append(new_labels)
            graph_documents.append(" ".join(doc))
        
        # Update node labels for next iteration
        node_labels = new_node_labels
        documents.extend(graph_documents)
    
    # Extract features using bag-of-words approach
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform(documents)
    
    # Convert to TF-IDF features
    tfidf = TfidfTransformer()
    feature_matrix = tfidf.fit_transform(bow_matrix).toarray()
    
    # Reshape to get graph-level features (each WL iteration adds a document per graph)
    num_graphs = len(nx_graphs)
    graph_features = np.zeros((num_graphs, feature_matrix.shape[1]))
    
    for i in range(wl_iterations + 1):
        start_idx = i * num_graphs
        end_idx = (i + 1) * num_graphs
        if start_idx < feature_matrix.shape[0]:
            for j in range(num_graphs):
                if start_idx + j < feature_matrix.shape[0]:
                    graph_features[j] += feature_matrix[start_idx + j]
    
    # Reduce dimensions if needed
    if embedding_dim < graph_features.shape[1]:
        pca = PCA(n_components=embedding_dim)
        graph_features = pca.fit_transform(graph_features)
    
    return graph_features

train_embeddings = custom_graph2vec(train_graphs_subset, wl_iterations=3, embedding_dim=32)
val_embeddings = custom_graph2vec(val_graphs_subset, wl_iterations=3, embedding_dim=32)
test_embeddings = custom_graph2vec(test_graphs_subset, wl_iterations=3, embedding_dim=32)

print(f'Train embeddings shape: {train_embeddings.shape}')
print(f'Validation embeddings shape: {val_embeddings.shape}')
print(f'Test embeddings shape: {test_embeddings.shape}')

Train embeddings shape: (700, 32)
Validation embeddings shape: (150, 32)
Test embeddings shape: (150, 32)


In [15]:
from graph2vec import Graph2Vec # Ensure this path is correct

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize Graph2Vec model
g2v_model = Graph2Vec(
    wl_iterations=2,
    use_node_attribute='x', # Use 'x' for QM9 atom features, or None for degrees
    dimensions=8,
    workers=4,
    down_sampling=0.0001,
    epochs=10,
    learning_rate=0.025,
    min_count=5,
    seed=42,
    erase_base_features=False,
)

# Fit the model on the training data (NetworkX graphs)
g2v_model.fit(train_graphs_subset)

# Get embeddings for the training set
train_embeddings = g2v_model.get_embedding()

# Infer embeddings for validation and test sets
val_embeddings = g2v_model.infer(val_graphs_subset)
test_embeddings = g2v_model.infer(test_graphs_subset)

# Convert to PyTorch tensors - these are your X data
x_train = torch.tensor(train_embeddings, dtype=torch.float32).to(device)
x_val = torch.tensor(val_embeddings, dtype=torch.float32).to(device)
x_test = torch.tensor(test_embeddings, dtype=torch.float32).to(device)

print(f'x_train shape: {x_train.shape}')
print(f'x_val shape: {x_val.shape}')
print(f'x_test shape: {x_test.shape}')

x_train shape: torch.Size([91581, 8])
x_val shape: torch.Size([19624, 8])
x_test shape: torch.Size([19626, 8])


In [16]:
# Indices for HOMO and LUMO in QM9 target properties
# HOMO is at index 2, LUMO is at index 3 (0-indexed)
target_indices = [2, 3]

# These are your Y data, selecting only HOMO and LUMO
y_train = torch.stack([data.y[0, target_indices] for data in train_pyg_subset]).to(device)
y_val = torch.stack([data.y[0, target_indices] for data in val_pyg_subset]).to(device)
y_test = torch.stack([data.y[0, target_indices] for data in test_pyg_subset]).to(device)

print(f'x_train shape: {x_train.shape}, y_train shape: {y_train.shape}')
print(f'x_val shape: {x_val.shape}, y_val shape: {y_val.shape}')
print(f'x_test shape: {x_test.shape}, y_test shape: {y_test.shape}')
print(f'x_train device: {x_train.device}, y_train device: {y_train.device}')

x_train shape: torch.Size([91581, 8]), y_train shape: torch.Size([91581, 2])
x_val shape: torch.Size([19624, 8]), y_val shape: torch.Size([19624, 2])
x_test shape: torch.Size([19626, 8]), y_test shape: torch.Size([19626, 2])
x_train device: cuda:0, y_train device: cuda:0


In [17]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error

class MultiTargetMLP(nn.Module):
    def __init__(self, input_dim=15, hidden_dims=[64, 128, 64], output_dim=14):
        super(MultiTargetMLP, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(dim))
            layers.append(nn.Dropout(0.2))
            prev_dim = dim
            
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)


In [18]:
# Create dataset and dataloader
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Initialize model and optimizer
model = MultiTargetMLP(input_dim=x_train.shape[1], output_dim=y_train.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.MSELoss()

# Training loop
n_epochs = 500
best_val_mse = float('inf')
patience = 50
counter = 0

for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    
    for X_batch, y_batch in train_loader:        
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, y_val).item()
    
    print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val MSE: {val_loss:.4f}")
    
    # Early stopping
    if val_loss < best_val_mse:
        best_val_mse = val_loss
        counter = 0
        # Save model
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# Load best model and evaluate on test set
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

with torch.no_grad():
    test_preds = model(x_test).cpu().numpy()
    test_mse = mean_squared_error(y_test.cpu().numpy(), test_preds)
    print(f"Test MSE with MLP: {test_mse:.4f}")

Epoch 1/500, Train Loss: 5.2913, Val MSE: 0.8989
Epoch 2/500, Train Loss: 1.0552, Val MSE: 0.8427
Epoch 3/500, Train Loss: 0.9105, Val MSE: 0.8617
Epoch 4/500, Train Loss: 0.8498, Val MSE: 0.8064
Epoch 5/500, Train Loss: 0.8265, Val MSE: 0.8744
Epoch 6/500, Train Loss: 0.8148, Val MSE: 0.8508
Epoch 7/500, Train Loss: 0.8060, Val MSE: 0.9049
Epoch 8/500, Train Loss: 0.8002, Val MSE: 0.9170
Epoch 9/500, Train Loss: 0.7965, Val MSE: 0.8136
Epoch 10/500, Train Loss: 0.7933, Val MSE: 0.8841
Epoch 11/500, Train Loss: 0.7868, Val MSE: 0.8127
Epoch 12/500, Train Loss: 0.7791, Val MSE: 0.8478
Epoch 13/500, Train Loss: 0.7737, Val MSE: 0.8956
Epoch 14/500, Train Loss: 0.7681, Val MSE: 0.8049
Epoch 15/500, Train Loss: 0.7609, Val MSE: 0.8128
Epoch 16/500, Train Loss: 0.7561, Val MSE: 0.8425
Epoch 17/500, Train Loss: 0.7535, Val MSE: 0.8334
Epoch 18/500, Train Loss: 0.7463, Val MSE: 0.8424
Epoch 19/500, Train Loss: 0.7423, Val MSE: 0.8604
Epoch 20/500, Train Loss: 0.7371, Val MSE: 0.8561
Epoch 21/